# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [21]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key


In [22]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ust'-uda,54.1742,103.0304,20.48,98,100,2.28,RU,1710194537
1,1,papatowai,-46.5619,169.4708,56.21,64,58,3.44,NZ,1710194421
2,2,nishinoomote,30.7333,131.0000,58.62,85,100,27.85,JP,1710194538
3,3,cockburn town,21.4612,-71.1419,78.67,74,0,8.32,TC,1710194538
4,4,lebu,-37.6167,-73.6500,62.20,72,0,17.34,CL,1710194401


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [23]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
lat_df = city_data_df['Lat']
lng_df = city_data_df['Lng']
city_df = city_data_df['City']
humidity_df = city_data_df['Humidity']

city_data_df1 = pd.DataFrame( { 
    'Latititude': lat_df,
    'Longtitude': lng_df,
    'City':city_df ,
    'Humidity':humidity_df})

map_plot_1 = city_data_df1.hvplot.points(
    'Longtitude',
    'Latititude',
    geo = True,
    size = 'Humidity',
    tiles = 'OSM',
    scale = 0.01
    )

# Display the map
# YOUR CODE HERE
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Longtitude,Latititude]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
city_data_df = city_data_df[(city_data_df['Max Temp'] > 22) &
(city_data_df['Max Temp'] < 30 ) & 
(city_data_df['Cloudiness'] <= 50)]
# Drop any rows with null values
# YOUR CODE HERE
city_data_df = city_data_df.dropna()

# Display sample data
# YOUR CODE HERE
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
121,121,razumnoye,50.5344,36.6846,26.67,63,0,4.47,RU,1710194552
201,201,liaoyuan,42.9036,125.1358,26.96,55,3,2.91,CN,1710194374
205,205,shitanjing,39.2342,106.3439,27.84,38,6,8.28,CN,1710194563
303,303,volokonovka,50.4827,37.8563,26.73,57,2,4.38,RU,1710194426
415,415,inari,68.9060,27.0288,25.00,97,41,4.45,FI,1710194595
542,542,alta,69.9689,23.2717,29.46,83,40,2.30,NO,1710194615
562,562,kapustin yar,48.5831,45.7372,24.35,70,3,7.87,RU,1710194618
573,573,bodo,67.2800,14.4050,26.64,83,0,5.75,NO,1710194527
588,588,fuyu,45.1833,124.8167,25.50,55,7,6.26,CN,1710194622


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ''
# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
121,razumnoye,RU,50.5344,36.6846,63,
201,liaoyuan,CN,42.9036,125.1358,55,
205,shitanjing,CN,39.2342,106.3439,38,
303,volokonovka,RU,50.4827,37.8563,57,
415,inari,FI,68.9060,27.0288,97,
542,alta,NO,69.9689,23.2717,83,
562,kapustin yar,RU,48.5831,45.7372,70,
573,bodo,NO,67.2800,14.4050,83,
588,fuyu,CN,45.1833,124.8167,55,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000 


params = { 'categories':'accommodation.hotel',
          "apiKey": geoapify_key, 
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except(KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
razumnoye - nearest hotel: Лето
liaoyuan - nearest hotel: No hotel found
shitanjing - nearest hotel: No hotel found
volokonovka - nearest hotel: Нива
inari - nearest hotel: Villa Lanca
alta - nearest hotel: Scandic Alta
kapustin yar - nearest hotel: Виктория
bodo - nearest hotel: Zefyr hotell
fuyu - nearest hotel: 松原天達名都大酒店


,City,Country,Lat,Lng,Humidity,Hotel Name
121,razumnoye,RU,50.5344,36.6846,63,Лето
201,liaoyuan,CN,42.9036,125.1358,55,No hotel found
205,shitanjing,CN,39.2342,106.3439,38,No hotel found
303,volokonovka,RU,50.4827,37.8563,57,Нива
415,inari,FI,68.9060,27.0288,97,Villa Lanca
542,alta,NO,69.9689,23.2717,83,Scandic Alta
562,kapustin yar,RU,48.5831,45.7372,70,Виктория
573,bodo,NO,67.2800,14.4050,83,Zefyr hotell
588,fuyu,CN,45.1833,124.8167,55,松原天達名都大酒店


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display
import random 
# Generate a random color for each city
city_colors = {}
for city in hotel_df['City'].unique():
    city_colors[city] = "#{:06x}".format(random.randint(0, 0xFFFFFF))
# Function to get the color for each city
def get_city_color(city):
    return city_colors.get(city, 'gray')
# Add a 'Color' column to the DataFrame based on the city
hotel_df['Color'] = hotel_df['City'].apply(get_city_color)
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols =["Hotel Name","Country"],
    geo = True,
    tiles = "OSM",
    color='Color'
)
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Color,Hotel Name,Country)